# Part 4. Matrix and Vector Computations

#### 1.Read about Broadcasting with Arrays on the chapter Computation on Arrays: Broadcasting from Python Data Science Handbook (J. VandePlas, 2016): Link: https://jakevdp.github.io/PythonDataScienceHandbook/02.03-computation-on-arrays-ufuncs.html

The section from the Python Data Science Handbook discusses the importance of vectorized operations in NumPy for efficient computation, particularly through universal functions (ufuncs). It explains the slowness of loops in Python and introduces ufuncs as a solution for faster array computations. 

The document covers unary and binary ufuncs, arithmetic and trigonometric operations, absolute values, exponents, and logarithms. It also explores specialized ufuncs, output specification, aggregates, and outer products, detailing how these features can enhance computation efficiency and capability in NumPy.

#### 2. Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high Performance. Second Edition. UK: Packt Publishing Ltd. 

#### Implement the improvements on the particle simulator using NumPy. Show that both implementations scale linearly with particle size, but the runtime in the pure Python version grows much faster than the NumPy version.

In [59]:
import numpy as np
from random import uniform
 
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel
 
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
 
    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
 
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y
 
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
 
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
 
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]
 
def benchmark(npart=100, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
 
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)
 
# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit
 
    print("Benchmarking with 100 particles:")
    print("Python version:", timeit("benchmark(100, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(100, 'numpy')", globals=globals(), number=1), "seconds")

Benchmarking with 100 particles:
Python version: 0.3331199989988818 seconds
NumPy version: 0.09139973599667428 seconds


#### 3. Explain how to optain the optimal performance with numexpr. Read the section. Reaching optimal performance with numexpr, pp. 72 from the previous reference. Implement it and measure the execution time.

NumPy accelerates calculations by utilizing array broadcasting and efficient memory use, which becomes particularly advantageous with large data sets, as demonstrated by benchmark comparisons between pure Python and NumPy implementations. 

The transition from Python loops to NumPy's vectorized operations results in noticeable speed-ups, particularly as the size of the data increases.

Numexpr extends these benefits by optimizing array expressions further, reducing memory usage, and utilizing multiple cores for parallel computation. 
By avoiding intermediate array storage and exploiting CPU cache efficiently, numexpr can significantly speed up operations involving large arrays. 

This is illustrated in the context of calculating norms and particle displacements where numexpr's capability to compile and optimize complex expressions on-the-fly shows substantial performance enhancements over standard NumPy operations.

In [58]:
import numpy as np
from random import uniform
import numexpr as ne

 
class Particle:
    def __init__(self, x, y, ang_vel):
        self.x = x
        self.y = y
        self.ang_vel = ang_vel
 
class ParticleSimulator:
    def __init__(self, particles):
        self.particles = particles
 
    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
 
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.ang_vel * v_x
                d_y = timestep * p.ang_vel * v_y
                p.x += d_x
                p.y += d_y
 
    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])
 
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * ang_vel_i[:, np.newaxis] * v_i
            r_i += d_i
 
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

    def evolve_numexpr(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        ang_vel_i = np.array([p.ang_vel for p in self.particles])

        for i in range(nsteps):
            norm_i = ne.evaluate("sqrt((r_i ** 2).sum(axis=1))")
            v_i = r_i[:, [1, 0]]
            v_i[:, 0] = ne.evaluate("-v_i[:, 0]")
            v_i = ne.evaluate("v_i / norm_i[:, None]")
            d_i = ne.evaluate("timestep * ang_vel_i[:, None] * v_i")
            r_i = ne.evaluate("r_i + d_i")

        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

def benchmark(npart=100, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
 
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)
    elif method == 'numexpr':
        simulator.evolve_numpy(0.1)

# Example of benchmarking
if __name__ == "__main__":
    from timeit import timeit
    
    print("Benchmarking with 1000 particles:")
    print("Python version:", timeit("benchmark(1000, 'python')", globals=globals(), number=1), "seconds")
    print("NumPy version:", timeit("benchmark(1000, 'numpy')", globals=globals(), number=1), "seconds")
    print("NumExpr version:", timeit("benchmark(1000, 'numexpr')", globals=globals(), number=1), "seconds")


Benchmarking with 1000 particles:
Python version: 2.957614521998039 seconds
NumPy version: 0.26057985499937786 seconds
NumExpr version: 0.2573370470054215 seconds
